In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
train_data = pd.read_csv('train.csv')
pred_data = pd.read_csv('test.csv')

In [4]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data.label.value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [7]:
X = train_data.drop('label', axis=1).copy()
y = train_data.label.copy()

In [8]:
X = tf.reshape(X, shape=[-1, 28, 28, 1])
X = tf.cast(X, tf.float32) / 255.

In [9]:
pred_data = tf.reshape(pred_data, shape=[-1, 28, 28, 1])
pred_data = tf.cast(pred_data, tf.float32) / 255.

In [10]:
X.shape, pred_data.shape

(TensorShape([42000, 28, 28, 1]), TensorShape([28000, 28, 28, 1]))

In [11]:
X_train, X_test = X[:-2000], X[-2000:]
y_train, y_test = y[:-2000], y[-2000:]

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
                                 keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu',input_shape=[28, 28, 1]),
                                 keras.layers.MaxPool2D(pool_size=2),
                                 keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
                                 keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
                                 keras.layers.MaxPool2D(pool_size=2),
                                 keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
                                 keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
                                 keras.layers.MaxPool2D(pool_size=2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(128, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(units=64, activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)         2

In [ ]:
model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.000001),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics='accuracy')

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
         validation_split=0.2)

Epoch 1/5
1000/1000 [==============================] - 36s 34ms/step - loss: 0.0103 - accuracy: 0.9973 - val_loss: 0.0448 - val_accuracy: 0.9933
Epoch 2/5
1000/1000 [==============================] - 24s 24ms/step - loss: 0.0087 - accuracy: 0.9981 - val_loss: 0.0449 - val_accuracy: 0.9933
Epoch 3/5
1000/1000 [==============================] - 24s 24ms/step - loss: 0.0061 - accuracy: 0.9987 - val_loss: 0.0453 - val_accuracy: 0.9933
Epoch 4/5
1000/1000 [==============================] - 24s 24ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.0454 - val_accuracy: 0.9933
Epoch 5/5
1000/1000 [==============================] - 24s 24ms/step - loss: 0.0150 - accuracy: 0.9956 - val_loss: 0.0453 - val_accuracy: 0.9933


In [ ]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 1s 9ms/step - loss: 0.0398 - accuracy: 0.9920


[0.0398201122879982, 0.9919999837875366]

In [ ]:
pred_prob = model.predict(pred_data)
y_pred = np.argmax(pred_prob, axis=1)

In [ ]:
out = pd.DataFrame({'ImageId': np.arange(1, len(y_pred) + 1), 'Label': y_pred})
out

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [ ]:
out.to_csv('result.csv', index=False)